In [1]:
import networkx as nx
import streamlit as st
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [5]:
G = nx.karate_club_graph()

# Create position dictionary using spring layout
pos = nx.spring_layout(G)
node_positions = {node: (x * 1000, y * 1000) for node, (x, y) in pos.items()}

# Example compounds with their edges
compounds = {
    "Compound A": [(0, 1), (1, 2), (2, 3)],
    "Compound B": [(2, 3), (3, 4), (4, 5)],
    "Compound C": [(5, 6), (6, 7), (7, 8)]
}

# Streamlit UI: allow the user to select which compound to view
# selected_compound = st.sidebar.selectbox("Select a compound:", ["All"] + list(compounds.keys()))
selected_compound = "Compound A"

# Create Plotly graph
edge_x = []
edge_y = []
for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.append(x0)
    edge_x.append(x1)
    edge_y.append(y0)
    edge_y.append(y1)

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines'
)

node_x = []
node_y = []
for node in G.nodes():
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        size=10,
        color='blue',
        line_width=2
    )
)

# Add hover functionality
node_trace.marker.color = 'blue'  # Default color
node_trace.hoverinfo = 'text'
node_trace.text = [f'Node {node}' for node in G.nodes()]

# Filter edges for the selected compound
# edges_to_display = []
# if selected_compound != "All":
#     compound_edges = compounds[selected_compound]
#     for edge in compound_edges:
#         if edge in G.edges() or (edge[1], edge[0]) in G.edges():
#             x0, y0 = pos[edge[0]]
#             x1, y1 = pos[edge[1]]
#             edges_to_display.append([x0, y0, x1, y1])

In [10]:
edges_to_display = []
if selected_compound != "All":
    compound_edges = compounds[selected_compound]
    for edge in compound_edges:
        if edge in G.edges() or (edge[1], edge[0]) in G.edges():
            x0, y0 = pos[edge[0]]
            x1, y1 = pos[edge[1]]
            edges_to_display.append([(x0, y0), (x1, y1)])

# Create edge traces for the selected compound
edge_trace_selected = go.Scatter(
    x=[x for x, y in edges_to_display], 
    y=[y for x, y in edges_to_display],
    line=dict(width=2, color='red'),
    hoverinfo='none',
    mode='lines'
)


In [11]:
edge_trace_selected

Scatter({
    'hoverinfo': 'none',
    'line': {'color': 'red', 'width': 2},
    'mode': 'lines',
    'x': [[-0.3780216858793844, 0.2584850672649352], [-0.334446405888577,
          -0.0031482779612144723], [-0.08333786718481971, 0.01144088805643723]],
    'y': [[-0.334446405888577, -0.0031482779612144723], [-0.08333786718481971,
          0.01144088805643723], [-0.4742720150049916, -0.008233988594917644]]
})

In [ ]:
viz_data = pd.read_excel("/Users/rebeccakrall/Desktop/Visualization.xlsx", skiprows =1 )
viz_header = pd.read_excel("/Users/rebeccakrall/Desktop/Visualization.xlsx", header = [0,1])
data_only = viz_data.iloc[:,5:-1]
column_cats = {k:v for v,k in viz_header.iloc[:,5:-1].columns}

lowest_dose = pd.DataFrame()
max_val = viz_data.iloc[:, -1]
for c in data_only.columns:
    if data_only[c].dtype != 'float64':
        vals = data_only[c].str.split(',', expand = True)[0]
        vals = vals.apply(lambda x: np.nan if 't' in str(x) else x)
    else:
        vals = data_only[c]
    dose = vals.astype(float)/max_val.astype(float)
    lowest_dose[c] = dose
lowest_dose = lowest_dose.dropna(axis = 1, how= 'all')

var_mapping = {}
for ind,col in enumerate(lowest_dose.columns):
    var_mapping[col] = ind